In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

In [2]:
data_file = r"C:\Users\krish\Downloads\archive (9)\hin.txt"  

english_sentences = []
hindi_sentences = []

with open(data_file, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split("\t")  # Tab-separated values
        if len(parts) >= 2:
            english_sentences.append(parts[0])  # English text
            hindi_sentences.append(parts[1])  # Hindi text 

In [3]:
eng_tokenizer = Tokenizer(filters='', oov_token="<OOV>")
hin_tokenizer = Tokenizer(filters='', oov_token="<OOV>")

eng_tokenizer.fit_on_texts(english_sentences)
hin_tokenizer.fit_on_texts(hindi_sentences)

eng_vocab_size = len(eng_tokenizer.word_index) + 1
hin_vocab_size = len(hin_tokenizer.word_index) + 1

eng_sequences = eng_tokenizer.texts_to_sequences(english_sentences)
hin_sequences = hin_tokenizer.texts_to_sequences(hindi_sentences)

max_eng_length = max(len(seq) for seq in eng_sequences)
max_hin_length = max(len(seq) for seq in hin_sequences)

eng_padded = pad_sequences(eng_sequences, maxlen=max_eng_length, padding="post")
hin_padded = pad_sequences(hin_sequences, maxlen=max_hin_length, padding="post")

hin_padded_cat = np.array([to_categorical(seq, num_classes=hin_vocab_size) for seq in hin_padded])

In [4]:
latent_dim = 256

In [5]:
def build_encoder():
    encoder_inputs = Input(shape=(max_eng_length,))
    enc_emb = Embedding(eng_vocab_size, latent_dim, mask_zero=True)(encoder_inputs)
    encoder_lstm = LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
    encoder_states = [state_h, state_c]
    
    return Model(encoder_inputs, encoder_states), encoder_inputs, encoder_states

In [6]:
def build_decoder(encoder_states):
    decoder_inputs = Input(shape=(max_hin_length,))
    
    # Embedding layer for decoder
    dec_emb_layer = Embedding(hin_vocab_size, latent_dim, mask_zero=True)
    dec_emb = dec_emb_layer(decoder_inputs)

    # LSTM layer
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, state_h, state_c = decoder_lstm(dec_emb, initial_state=encoder_states)

    # Dense output layer
    decoder_dense = Dense(hin_vocab_size, activation="softmax")
    decoder_outputs = decoder_dense(decoder_outputs)

    # Training model (outputs full sequence)
    decoder_model_train = Model([decoder_inputs] + encoder_states, decoder_outputs)

    # Inference model for decoding one step at a time
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_hidden_state_input = Input(shape=(max_eng_length, latent_dim))  # Encoder output

    dec_emb_inf = dec_emb_layer(decoder_inputs)
    decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(
        dec_emb_inf, initial_state=[decoder_state_input_h, decoder_state_input_c]
    )
    decoder_outputs_inf = decoder_dense(decoder_outputs_inf)

    decoder_model_inf = Model(
        [decoder_inputs] + [decoder_state_input_h, decoder_state_input_c, decoder_hidden_state_input],
        [decoder_outputs_inf, state_h_inf, state_c_inf]
    )

    return decoder_model_train, decoder_model_inf, decoder_inputs


In [7]:
encoder_model, encoder_inputs, encoder_states = build_encoder()
decoder_model_train, decoder_model_inf, decoder_inputs = build_decoder(encoder_states)
model = Model([encoder_inputs, decoder_inputs], decoder_model_train([decoder_inputs] + encoder_states))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit([eng_padded, hin_padded], hin_padded_cat, batch_size=64, epochs=40, validation_split=0.1)

Epoch 1/40
40/40 [==============================] - 26s 398ms/step - loss: 7.2504 - accuracy: 0.0750 - val_loss: 6.8206 - val_accuracy: 0.0369
Epoch 2/40
40/40 [==============================] - 13s 330ms/step - loss: 6.1371 - accuracy: 0.0710 - val_loss: 6.4513 - val_accuracy: 0.0441
Epoch 3/40
40/40 [==============================] - 12s 290ms/step - loss: 5.6079 - accuracy: 0.0795 - val_loss: 6.1812 - val_accuracy: 0.0739
Epoch 4/40
40/40 [==============================] - 12s 302ms/step - loss: 5.2595 - accuracy: 0.1552 - val_loss: 5.9137 - val_accuracy: 0.1438
Epoch 5/40
40/40 [==============================] - 13s 320ms/step - loss: 4.8523 - accuracy: 0.2306 - val_loss: 5.5322 - val_accuracy: 0.2393
Epoch 6/40
40/40 [==============================] - 12s 310ms/step - loss: 4.3701 - accuracy: 0.3222 - val_loss: 5.0911 - val_accuracy: 0.3109
Epoch 7/40
40/40 [==============================] - 13s 321ms/step - loss: 3.8963 - accuracy: 0.4062 - val_loss: 4.6686 - val_accuracy: 0.3929

In [8]:
model.save("eng_hin_translation.h5")
import pickle
with open("eng_tokenizer.pkl", "wb") as f:
    pickle.dump(eng_tokenizer, f)
with open("hin_tokenizer.pkl", "wb") as f:
    pickle.dump(hin_tokenizer, f)

C:\Users\krish\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
loss, accuracy = model.evaluate([eng_padded, hin_padded], hin_padded_cat)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

87/87 [==============================] - 7s 79ms/step - loss: 0.2669 - accuracy: 0.9733
Test Loss: 0.26686930656433105, Test Accuracy: 0.9733326435089111
